This is a notebook made for AWS SageMaker Studio.

### Background
This notebook has been adapted from an AWS blog post.

This notebook uses machine learning (ML) for the automated identification of unhappy customers, also known as customer churn prediction. It also shows how to incorporate the relative costs of prediction mistakes when determining the financial outcome of using ML.

Example of churn: leaving a mobile phone operator. If the provider knows that a customer is thinking of leaving, it can offer timely incentives - such as a phone upgrade or perhaps having a new feature activated – and the customer may stick around. Incentives are often much more cost-effective than losing and reacquiring a customer.

In [3]:
import sys
!{sys.executable} -m pip install sagemaker pandas numpy --upgrade

In [ ]:
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
prefix = "sagemaker/DEMO-xgboost-churn"

# Define IAM role
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()

In [1]:
# Parameters

# KMS key - not needed for this project
# kms_key = ""

Import libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import os
import sys
import time
import json
from IPython.display import display
from time import strftime, gmtime
from sagemaker.inputs import TrainingInput
from sagemaker.serializers import CSVSerializer

In [ ]:
# get the file
s3 = boto3.client("s3")
s3.download_file(f"sagemaker-sample-files", "datasets/tabular/synthetic/churn.txt", "churn.txt")

In [ ]:
churn = pd.read_csv("./churn.txt")
pd.set_option("display.max_columns", 500)
churn

In [ ]:
len(churn.columns)

### Explore the data

In [ ]:
# Frequency tables for each categorical feature
for column in churn.select_dtypes(include=["object"]).columns:
    display(pd.crosstab(index=churn[column], columns="% observations", normalize="columns"))

# Histograms for each numeric features
display(churn.describe())
%matplotlib inline
hist = churn.hist(bins=30, sharey=True, figsize=(10, 10))

- State - be quite evenly distributed. 
- Phone - too many unique values to be useful. It’s possible that parsing out the prefix could have some value, but without more context on how these are allocated, avoid using it. 
- Most of the numeric features are surprisingly nicely distributed. 
  - VMail Message - exception 
  - Area Code - should be converted to non-numeric

In [ ]:
churn = churn.drop("Phone", axis=1)
churn["Area Code"] = churn["Area Code"].astype(object)

Relationship to the target

In [ ]:
for column in churn.select_dtypes(include=["object"]).columns:
    if column != "Churn?":
        display(pd.crosstab(index=churn[column], columns=churn["Churn?"], normalize="columns"))

for column in churn.select_dtypes(exclude=["object"]).columns:
    print(column)
    hist = churn[[column, "Churn?"]].hist(by="Churn?", bins=30)
    plt.show()

In [ ]:
display(churn.corr())
pd.plotting.scatter_matrix(churn, figsize=(12, 12))
plt.show()

- some features have ~100% correlation with one another 
  - remove one feature from each of the highly correlated pairs
    - Day Charge from the pair with Day Mins and so on

In [ ]:
churn = churn.drop(["Day Charge", "Eve Charge", "Night Charge", "Intl Charge"], axis=1)

In [ ]:
model_data = pd.get_dummies(churn)
model_data = pd.concat(
    [model_data["Churn?_True."], model_data.drop(["Churn?_False.", "Churn?_True."], axis=1)], axis=1
)

Amazon SageMaker provides an XGBoost container to train in a managed, distributed setting, and then host as a real-time prediction endpoint.

Split data.

In [ ]:
train_data, validation_data, test_data = np.split(
    model_data.sample(frac=1, random_state=1729),
    [int(0.7 * len(model_data)), int(0.9 * len(model_data))],
)
train_data.to_csv("train.csv", header=False, index=False)
validation_data.to_csv("validation.csv", header=False, index=False)

In [5]:
len(train_data.columns)

Upload to S3

In [ ]:
boto3.Session().resource("s3").Bucket(bucket).Object(
    os.path.join(prefix, "train/train.csv")
).upload_file("train.csv")
boto3.Session().resource("s3").Bucket(bucket).Object(
    os.path.join(prefix, "validation/validation.csv")
).upload_file("validation.csv")

### Training

In [ ]:
container = sagemaker.image_uris.retrieve("xgboost", sess.boto_region_name, "1.7-1")

Create TrainingInputs that our training function can use as a pointer to the files in S3.

In [ ]:
s3_input_train = TrainingInput(
    s3_data="s3://{}/{}/train".format(bucket, prefix), content_type="csv"
)
s3_input_validation = TrainingInput(
    s3_data="s3://{}/{}/validation/".format(bucket, prefix), content_type="csv"
)

Hyperparameters:
 - max_depth - how deep each tree within the algorithm can be built. Deeper trees can lead to better fit, but are more computationally expensive and can lead to overfitting
 - subsample -sampling of the training data. (reduces overfitting, but setting it too low can also starve the model of data)
 - num_round - the number of boosting rounds (the subsequent models that are trained using the residuals of previous iterations)
 - eta - how aggressive each round of boosting is (Larger values lead to more conservative boosting)
 - gamma - how aggressively trees are grown. (Larger values lead to more conservative models)

In [ ]:
sess = sagemaker.Session()

xgb = sagemaker.estimator.Estimator(
    container,
    role,
    instance_count=1,
    instance_type="ml.m4.xlarge",
    output_path="s3://{}/{}/output".format(bucket, prefix),
    sagemaker_session=sess,
)
xgb.set_hyperparameters(
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.8,
    verbosity=0,
    objective="binary:logistic",
    num_round=100,
)

In [ ]:
xgb.fit({"train": s3_input_train, "validation": s3_input_validation})

### Create a model and deploy it to a hosted endpoint.

In [ ]:
xgb_predictor = xgb.deploy(
    initial_instance_count=1, instance_type="ml.m4.xlarge", serializer=CSVSerializer()
)

### Evaluate

function: 
1. Loop over our test dataset 
1. Split it into mini-batches of rows 
1. Convert those mini-batchs to CSV string payloads 
1. Retrieve mini-batch predictions by invoking the XGBoost endpoint 
1. Collect predictions and convert from the CSV output our model provides into a NumPy array

In [ ]:
def predict(data, rows=500):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ""
    for array in split_array:
        predictions = ",".join([predictions, xgb_predictor.predict(array).decode("utf-8")])

    return np.fromstring(predictions[1:], sep=",")


predictions = predict(test_data.to_numpy()[:, 1:])

In [ ]:
print(predictions)

Because of the np.round() function, the threshold is 0.5.

In [ ]:
pd.crosstab(
    index=test_data.iloc[:, 0],
    columns=np.round(predictions),
    rownames=["actual"],
    colnames=["predictions"],
)

In [ ]:
plt.hist(predictions)
plt.xlabel("Predicted churn probability")
plt.ylabel("Number of customers")
plt.show()

Try other thresholds

In [ ]:
threshold = 0.3
pd.crosstab(index=test_data.iloc[:, 0], columns=np.where(predictions > threshold, 1, 0))

### Finding the optimal threshold

In [ ]:
cutoffs = np.arange(0.01, 1, 0.01)
costs = []
for c in cutoffs:
    costs.append(
        np.sum(
            np.sum(
                np.array([[0, 100], [500, 100]])
                * pd.crosstab(index=test_data.iloc[:, 0], columns=np.where(predictions > c, 1, 0))
            )
        )
    )

costs = np.array(costs)
plt.plot(cutoffs, costs)
plt.xlabel("Cutoff")
plt.ylabel("Cost")
plt.show()

In [ ]:
print(
    "Cost is minimized near a cutoff of:",
    cutoffs[np.argmin(costs)],
    "for a cost of:",
    np.min(costs),
)

### Cleanup

In [ ]:
xgb_predictor.delete_endpoint()